In [2]:
cd /home/jovyan/Documents/dsi_plus/Projects/project_3 - Madelon_feature_selection

/home/jovyan/Documents/dsi_plus/Projects/project_3 - Madelon_feature_selection


In [3]:
#Import standard modules
%run lib/__init__.py
%matplotlib inline

### Project 3: Madelon Dataset - Feature Selection + Classification

### Step 2. Feature Selection - RFE (Recursive Feature elimination)

Hyperparamters to tune:
    * Estimator: LogisticRegression - estimator needs to provide feature importance or coeff
        * penalty=l1 . ( l1 regularisation the allows coeffs to go to zero)
        * C  - select range of c, small c= high regularisation
    * n-features_to_select : No of features to select
    


In [4]:
#Load files names for datasets
pickle_files=Pickle_files()
datafiles=pickle_files.read_datafiles()

In [5]:
#List of classifiers to use for scoring selection
Classifiers={'RFE_KNN':KNeighborsClassifier(),'RFE_LR': LogisticRegression(C=100),\
             'RFE_DT': DecisionTreeClassifier(), 'RFE_SVC':SVC()}
XClassifiers={'RFE_LR': LogisticRegression(C=100)}

In [6]:
def rfe_pipeline(ds,Classifiers, paramG, datafiles=datafiles ):
    
    X=pd.read_pickle(datafiles[ds][2])
    y=pd.read_pickle(datafiles[ds][4])
    print(datafiles[ds][2],X.shape, datafiles[ds][4],y.shape)
    
    
    results=[]
    
    for n,model in tqdm(Classifiers.items()):   
       
        
        pipe=Pipeline(steps=[
                                ('scaler' , StandardScaler()),
                                ('rfe' , RFE(LogisticRegression(penalty='l1'))),
                                ('clf' ,model)
                               ])
    
        param=paramG

        grsCV=GridSearchCV(pipe,param,n_jobs=-1, verbose=1,cv=2)
        
        grsCV.fit(X,y.squeeze())
        
        feat_flag=grsCV.best_estimator_.named_steps['rfe'].get_support()
        
        
        results_n={ 'dataset' : ds,
                    'model'  : n,
                    'best_test_score' : grsCV.best_score_,
                    'best_params' : grsCV.best_params_ , 
                    'featSelect' : np.where(feat_flag)[0], 
                    'n_featSelect' : np.sum(feat_flag)
                    }
        results.append(results_n)
    
        
    results_df=pd.DataFrame(results)
    
    return results_df

#### UCI Dataset

In [7]:
UCI_param=({'rfe__n_features_to_select': [5,10],
            'rfe__estimator':[LogisticRegression(C=c) for c in [10,100,1000]]})

In [8]:
%%time
UCI_results_df=rfe_pipeline('UCI',Classifiers, UCI_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/UCI_X_train (2080, 500) ./data/pickle_data/UCI_y_train (2080, 1)
Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 10.0min finished
 25%|██▌       | 1/4 [10:45<32:15, 645.30s/it]

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 22.0min finished
 50%|█████     | 2/4 [33:34<33:34, 1007.20s/it]

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 10.1min finished
 75%|███████▌  | 3/4 [44:28<14:49, 889.61s/it] 

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 10.0min finished
100%|██████████| 4/4 [55:15<00:00, 828.96s/it]

CPU times: user 3min 8s, sys: 2.76 s, total: 3min 10s
Wall time: 55min 15s


In [9]:
UCI_results_df

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,"{'rfe__estimator': LogisticRegression(C=100, c...",0.849038,UCI,"[28, 48, 105, 128, 153, 241, 338, 378, 433, 451]",RFE_KNN,10
1,"{'rfe__estimator': LogisticRegression(C=100, c...",0.597596,UCI,"[28, 48, 105, 128, 153, 241, 338, 378, 433, 451]",RFE_LR,10
2,"{'rfe__estimator': LogisticRegression(C=1000, ...",0.778365,UCI,"[28, 48, 105, 128, 153, 241, 338, 378, 433, 451]",RFE_DT,10
3,"{'rfe__estimator': LogisticRegression(C=100, c...",0.802404,UCI,"[28, 48, 105, 128, 153, 241, 338, 378, 433, 451]",RFE_SVC,10


In [10]:
UCI_results_df.to_pickle('./data/pickle_data/UCI_bestfeatures_rfe')

#### Sample 1

In [14]:
Sample1_param=({'rfe__n_features_to_select': [5,10],
            'rfe__estimator':[LogisticRegression(C=c) for c in [10,100,1000]]})

In [ ]:
%%time
Sample1_results_df =rfe_pipeline('Sample1',Classifiers, Sample1_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/Sample1_X_train (7364, 1000) ./data/pickle_data/Sample1_y_train (7364,)
Fitting 2 folds for each of 6 candidates, totalling 12 fits


In [ ]:
Sample1_results_df 

#### Sample 2

In [ ]:
Sample2_param=({'rfe__n_features_to_select': [10],
            'rfe__estimator':[LogisticRegression(C=c) for c in [100,1000]]})

In [ ]:
%%time
Sample2_results_df =rfe_pipeline('Sample2',Classifiers, Sample2_param)

In [ ]:
Sample2_results_df 

#### Sample 3



In [ ]:
%%time
Sample3_results_df =rfe_pipeline('Sample3',Classifiers, Sample2_param)

In [ ]:
Sample3_results_df 

#### Consolidate results for samples

In [ ]:
Sample_results_df=pd.concat([Sample1_results_df,Sample2_results_df,\
                          Sample3_results_df], axis=0, ignore_index=True)

In [ ]:
Sample_results_df.to_pickle('./data/pickle_data/Samples_bestfeatures_rfe')